In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import scipy.io
from scipy.signal import butter, lfilter
import scipy.signal

In [2]:
# Load DEAP dataset (Change path accordingly)
data_path = "/kaggle/input/deap-matlab/"
subject_data = []
subject_labels = []

# Load all 32 subjects
for i in range(1, 33):
    mat = scipy.io.loadmat(f"{data_path}s{i:02d}.mat")
    
    # Extract EEG data and labels
    subject_data.append(mat["data"])     # Shape: (40 trials, 40 channels, 8064 samples)
    subject_labels.append(mat["labels"]) # Shape: (40 trials, 4 labels)

# Convert lists to NumPy arrays
subject_data = np.array(subject_data)     # Expected Shape: (32, 40, 40, 8064)
subject_labels = np.array(subject_labels) # Expected Shape: (32, 40, 4)

# Print shapes to confirm
print("EEG Data Shape:", subject_data.shape)   # (32, 40, 40, 8064)
print("Labels Shape:", subject_labels.shape)   # (32, 40, 4)


EEG Data Shape: (32, 40, 40, 8064)
Labels Shape: (32, 40, 4)


In [3]:
import numpy as np
import scipy.signal
# Frequency band definitions
freq_bands = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 14),
    "beta": (14, 30),
    "gamma": (31, 50),
}

# Define function to compute Differential Entropy (DE)
def compute_de(signal):
    """Compute Differential Entropy (DE) for a given EEG segment"""
    variance = np.var(signal, axis=-1, keepdims=True)  # Compute variance
    de = 0.5 * np.log(2 * np.pi * np.e * variance)  # Apply DE formula
    return de.squeeze()  # Remove extra dimensions

# Define function to extract DE features
def extract_de_features(subject_data, fs=128, window_size=128):
    """
    Extract DE features from EEG data.
    - subject_data: EEG data of shape (32, 40, 40, 8064)
    - fs: Sampling frequency (128 Hz)
    - window_size: 1 second (128 samples)
    Returns: DE feature array of shape (32, 40, 40, 5, 63)
    """
    num_subjects, num_trials, num_channels, num_samples = subject_data.shape
    num_bands = len(freq_bands)
    num_windows = num_samples // window_size  # 8064 / 128 = 63 windows

    # Initialize DE feature array
    de_features = np.zeros((num_subjects, num_trials, num_channels, num_bands, num_windows))

    # Loop through subjects, trials, and channels
    for subj in range(num_subjects):
        for trial in range(num_trials):
            for ch in range(num_channels):
                # Extract single-channel EEG data for this trial
                signal = subject_data[subj, trial, ch, :]

                # Apply bandpass filters and compute DE for each frequency band
                for b_idx, (band, (low, high)) in enumerate(freq_bands.items()):
                    # Bandpass filter
                    sos = scipy.signal.butter(4, [low, high], btype="bandpass", fs=fs, output="sos")
                    filtered_signal = scipy.signal.sosfilt(sos, signal)

                    # Segment into 1-second windows (128 samples each)
                    segmented = np.array(np.split(filtered_signal, num_windows, axis=-1))

                    # Compute DE for each window
                    de_features[subj, trial, ch, b_idx, :] = compute_de(segmented)

    return de_features

# Extract DE features
de_features = extract_de_features(subject_data)

# Print shape to confirm
print("DE Feature Shape:", de_features.shape)  # Expected: (32, 40, 40, 5, 63)

DE Feature Shape: (32, 40, 40, 5, 63)


In [4]:
1. Common Feature Extractor (CFE)
class CommonFeatureExtractor(nn.Module):
    def __init__(self, input_dim=200, output_dim=64):
        super(CommonFeatureExtractor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, output_dim)
        self.activation = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.activation(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.activation(self.bn2(self.fc2(x)))
        x = self.fc3(x)
        return x

In [5]:
# 2. Contrastive Loss L_con1
class ContrastiveLossLcon1(nn.Module):
    def __init__(self, tau=0.2):
        super(ContrastiveLossLcon1, self).__init__()
        self.tau = tau

    def forward(self, q):
        batch_size = q.shape[0]
        q = F.normalize(q, dim=-1)
        sim_matrix = torch.mm(q, q.T)  
        mask = torch.eye(batch_size, dtype=torch.bool, device=q.device)
        sim_matrix.masked_fill_(mask, -5.0)  
        if labels is None:
            exp_sim = torch.exp(sim_matrix / self.tau)
            loss = -torch.log(exp_sim / (exp_sim.sum(dim=-1, keepdim=True) + 1e-9)).mean()
        # Supervised contrastive loss (labels provided)
        else:
            exp_sim = torch.exp(sim_matrix / self.tau)
            mask_same_class = labels.unsqueeze(1) == labels.unsqueeze(0)
            exp_sim_same = exp_sim * mask_same_class.float()  
            loss = -torch.log(exp_sim_same.sum(dim=-1) / (exp_sim.sum(dim=-1) + 1e-9)).mean()

        return loss

In [6]:
# 3. Subject-Specific Feature Extractor (SFE)

class SubjectSpecificFeatureExtractor(nn.Module):
    def __init__(self, input_dim=64, output_dim=32):
        super(SubjectSpecificFeatureExtractor, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = nn.LeakyReLU()

    def forward(self, x):
        return self.activation(self.fc(x))  # Output shape: (batch_size, 32)

In [7]:
#4. Maximum Mean Discrepancy (MMD) Loss
def mmd_loss(source_features, target_features):
    source_mean = source_features.mean(dim=0)
    target_mean = target_features.mean(dim=0)
    loss = torch.norm(source_mean - target_mean, p=2) ** 2  
    return loss

In [8]:
# 5. Contrastive Loss L_con2 with Class Prototypes
class ContrastiveLossLcon2(nn.Module):
    def __init__(self, feature_dim=32, num_classes=4, tau=0.3, gamma=0.5, queue_size=1024):
        super(ContrastiveLossLcon2, self).__init__()
        self.tau = tau
        self.gamma = gamma
        self.num_classes = num_classes
        self.queue_size = queue_size

        # Initialize class prototypes (μ_c)
        self.prototypes = nn.Parameter(torch.randn(num_classes, feature_dim))  # Shape: (4, 32)

        # Initialize memory queue for negative samples
        self.queue = torch.randn(queue_size, feature_dim)  # Shape: (1024, 32)
        self.queue = F.normalize(self.queue, dim=-1)  # Normalize queue embeddings

    def forward(self, z_t, pseudo_labels):
        """
        Compute contrastive loss L_con2 for inter-domain alignment.
        - z_t: Target domain features (batch_size, trials, time_windows, 32)
        - pseudo_labels: Pseudo-labels for target samples (batch_size, trials, time_windows)

        Returns:
            Contrastive loss scalar
        """
        batch_size, trials, time_windows, feature_dim = z_t.shape

        # Flatten input for processing
        z_t = z_t.view(-1, feature_dim)  # Shape: (batch_size * trials * time_windows, 32)
        pseudo_labels = pseudo_labels.view(-1)  # Shape: (batch_size * trials * time_windows)

        # Normalize embeddings
        z_t = F.normalize(z_t, dim=-1)  # Normalize target embeddings
        self.prototypes.data = F.normalize(self.prototypes.data, dim=-1)  # Normalize prototypes

        # Compute similarity to class prototypes
        similarity = torch.mm(z_t, self.prototypes.T)  # Shape: (batch_size * trials * time_windows, 4)
        
        # Select correct class prototype based on pseudo-labels
        proto_sim = similarity.gather(1, pseudo_labels.unsqueeze(1))  # Shape: (batch_size * trials * time_windows, 1)

        # Compute softmax denominator (all possible embeddings)
        queue_sim = torch.mm(z_t, self.queue.T)  # Shape: (batch_size * trials * time_windows, queue_size)
        exp_sim = torch.cat([proto_sim, queue_sim], dim=1)  # Concatenate prototypes & queue
        exp_sim = torch.exp(exp_sim / self.tau)  # Apply temperature scaling

        # Contrastive loss computation
        loss = -torch.log(exp_sim[:, 0] / exp_sim.sum(dim=1))  # Only consider prototype similarity
        loss = loss.mean()

        # Update prototypes with momentum (γ)
        for i in range(self.num_classes):
            class_mask = (pseudo_labels == i).float().unsqueeze(1)  # Mask for samples of class i
            class_mean = (class_mask * z_t).sum(dim=0) / (class_mask.sum() + 1e-9)  # Compute class mean
            self.prototypes.data[i] = F.normalize(self.gamma * self.prototypes.data[i] + (1 - self.gamma) * class_mean, dim=-1)

        # Update memory queue (FIFO replacement)
        self.queue = torch.cat([self.queue[batch_size:], z_t.detach()], dim=0)  # Remove old, add new

        return loss

In [9]:
# 6. Subject-Specific Classifier (SSC)
class SubjectSpecificClassifier(nn.Module):
    def __init__(self, input_dim=32, num_classes=4):
        super(SubjectSpecificClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)  

In [10]:
# 7. Generalized Cross-Entropy (GCE) Loss
class GCELoss(nn.Module):
    def __init__(self, q=0.55):
        super(GCELoss, self).__init__()
        self.q = q

    def forward(self, logits, targets):
        probs = F.softmax(logits, dim=-1)
        true_probs = probs.gather(1, targets.unsqueeze(1)).squeeze()
        loss = (1 - true_probs ** self.q) / self.q
        return loss.mean()

In [ ]:
de_features = torch.tensor(de_features, dtype=torch.float32)  # (32, 40, 40, 5, 63)
num_subjects, num_trials, num_channels, num_bands, num_windows = de_features.shape
de_features = de_features.view(num_subjects, num_trials, num_windows, num_channels * num_bands)  # (32, 40, 63, 200)
print("Final Shape:", de_features.shape)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cfe = CommonFeatureExtractor().to(device)
sfe = SubjectSpecificFeatureExtractor().to(device)
ssc = SubjectSpecificClassifier().to(device)
contrastive_loss_lcon1 = ContrastiveLossLcon1().to(device)
contrastive_loss_lcon2 = ContrastiveLossLcon2().to(device)
gce_loss_fn = GCELoss().to(device)

# Optimizer
optimizer = optim.Adam(list(cfe.parameters()) + list(sfe.parameters()) + list(ssc.parameters()), lr=0.01)

# Training settings
epochs = 50
batch_size = 32  # Matches subjects in dataset


for epoch in range(epochs):
    optimizer.zero_grad()

    # Reshape DE features for CFE input
    num_subjects, num_trials, num_windows, feature_dim = de_features.shape  # (32, 40, 63, 200)

    # Reshape into (batch_size * trials * time_windows, 200)
    reshaped_features = de_features.reshape(-1, feature_dim)
    reshaped_features = reshaped_features.to(device)

    print("Final Shape:", de_features.shape)  # Expected: (32 * 40 * 63, 200)

    # Forward pass through CFE
    shared_features = cfe(reshaped_features)  # (batch_size, 64)
    common_loss = contrastive_loss_lcon1(shared_features)

    # Forward pass through SFE
    subject_features = sfe(shared_features)  # (batch_size, 32)

    # Compute MMD loss (iterations < 420)
    if epoch < 420:
        mmd_value = mmd_loss(shared_features, subject_features)
    else:
        mmd_value = torch.tensor(0.0)

    # Forward pass through SSC
    predictions = ssc(subject_features)  # (batch_size, 4)

    # Compute GCE Loss
    labels = torch.randint(0, 4, (batch_size,))
    gce_loss = gce_loss_fn(predictions, labels)

    # Compute L_con2
    pseudo_labels = torch.randint(0, 4, (batch_size,))
    lcon2_value = contrastive_loss_lcon2(subject_features, pseudo_labels)

    # Final Loss
    total_loss = gce_loss + mmd_value + common_loss + lcon2_value

    # Backpropagation
    total_loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}/{epochs} - Total Loss: {total_loss.item():.4f}")

print("✅ MSCL Training Complete!")

Final Shape: torch.Size([32, 40, 63, 200])


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.23 GiB. GPU 0 has a total capacity of 14.74 GiB of which 13.89 GiB is free. Process 5515 has 872.00 MiB memory in use. Of the allocated memory 729.07 MiB is allocated by PyTorch, and 16.93 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)